In [ ]:
# This is a jnb script run to generate file `statics/cif/mapping.json`.
# Check README for the reason why configurations are choose for different verification workflows.

In [ ]:
import json
from pymatgen.io.cif import CifParser
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import os

from aiida_sssp_workflow.utils import (
    MAGNETIC_ELEMENTS,
    ACTINIDE_ELEMENTS,
    RARE_EARTH_ELEMENTS,
    NO_GS_CONF_ELEMENTS,
)

In [ ]:
# The following is the list of elements that are used in the SSSP workflow.
elements = (
    [
        "Ag",
        "Al",
        "Ar",
        "As",
        "At",
        "Au",
        "Ba",
        "Be",
        "B",
        "Bi",
        "Br",
        "Ca",
        "Cd",
        "Ce",
        "C",
        "Cl",
        "Co",
        "Cr",
        "Cs",
        "Cu",
        "Dy",
        "Er",
        "Eu",
        "Fe",
        "F",
        "Ga",
        "Gd",
        "Ge",
        "He",
        "Hf",
        "H",
        "Hg",
        "Ho",
        "I",
        "In",
        "Ir",
        "K",
        "Kr",
        "La",
        "Li",
        "Lu",
        "Mg",
        "Mn",
        "Mo",
        "Na",
        "Nb",
        "Nd",
        "Ne",
        "N",
        "Ni",
        "O",
        "Os",
        "Pb",
        "Pd",
        "P",
        "Pm",
        "Po",
        "Pr",
        "Pt",
        "Rb",
        "Re",
        "Rh",
        "Rn",
        "Ru",
        "Sb",
        "Sc",
        "Se",
        "S",
        "Si",
        "Sm",
        "Sn",
        "Sr",
        "Ta",
        "Tb",
        "Tc",
        "Te",
        "Ti",
        "Tl",
        "Tm",
        "V",
        "W",
        "Xe",
        "Yb",
        "Y",
        "Zn",
        "Zr",
    ]
    + ACTINIDE_ELEMENTS
    + NO_GS_CONF_ELEMENTS
)

In [ ]:
cif_gs_folder = os.path.abspath("./structures/gs/")

sgn_t = {
    "229": "BCC",
    "225": "FCC",
    "221": "SC",
    "227": "Diamond",
    "194": "HCP",
}
gs_mapping = {}
for element in elements:
    try:
        cifparser = CifParser(filename=os.path.join(cif_gs_folder, f"{element}.cif"))
    except FileNotFoundError:
        print(f"{element} not has gs strucuture cif.")
        continue

    structure = cifparser.get_structures(primitive=True, symmetrized=False)[0]
    sga = SpacegroupAnalyzer(structure=structure)
    sgn = sga.get_space_group_number()
    gs_mapping[element] = sgn_t.get(str(sgn), "NaN")
    print(f"{element}: {sgn_t.get(str(sgn), 'NaN')}")

# print(gs_mapping)

In [ ]:
d = {}
d[
    "_commont"
] = "The mapping from element to configuration for convergence and bands verification."

import os
from aiida_sssp_workflow.utils import RARE_EARTH_ELEMENTS
from aiida_sssp_workflow.utils import ACTINIDE_ELEMENTS

for e in elements:
    if e in NO_GS_CONF_ELEMENTS:
        # we don't have At in typical
        d[e] = {
            "bands": f"FCC",
            "convergence": f"FCC",
        }
        continue

    if e in ACTINIDE_ELEMENTS:
        # we don't have Actinides in typical
        # The paper "Dissertation, Philipps-Universität Marburg, 2022. DOI: 10.13140/RG.2.2.28627.25121"
        # use FCC for all actinides.
        d[e] = {
            "bands": f"FCC",
            "convergence": f"FCC",
        }
        continue

    if e in RARE_EARTH_ELEMENTS:
        d[e] = {
            "bands": f"RE",
            "convergence": f"RE",
        }
        continue

    d[e] = {
        "bands": f"GS",
        "convergence": f"GS",
    }

In [ ]:
with open("./structures/mapping.json", "w") as f:
    json.dump(d, f, indent=4)